手动实现卷积层、池化层、全连接层，对于简单理解这些层的原理特别好用

但是你懂得，还是已经打包好了的函数放心好用

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())
import numpy as np

2.0.0
True


### 手动实现卷积层
可以与自带tf.keras.layers.Conv2D比较一下

In [2]:
inputs_ = np.random.random((10,28,28,3))#输入张量
w = np.random.random((6,3,3,3))#卷积核
b = np.random.random((6,))#每个通道的偏移量
pad=1 ##padding
stride=(1,1) #卷积核每次移动的步长

In [3]:
x = tf.constant(inputs_)
# w_tf = tf.constant(w)
# b_tf = tf.constant(b)

x = tf.dtypes.cast(x,dtype=tf.float32)
# b_tf = tf.dtypes.cast(w_tf,dtype=tf.float32)
# b_tf = tf.dtypes.cast(b_tf,dtype=tf.float32)

In [4]:
def corr2d(x, w, b, pad, stride):
    N, H, W, C = tf.shape(x)
    F, HH, WW, C = tf.shape(w)

    x = tf.pad(x, ((0, 0),(pad, pad), (pad, pad), (0, 0) ), 'constant')
    Hn = 1 + int((H + 2 * pad - HH) / stride[0])
    Wn = 1 + int((W + 2 * pad - WW) / stride[1])
    Y = tf.Variable(tf.zeros((N, Hn, Wn, F),dtype=tf.float32))

    for m in range(F):
        for i in range(Hn):
            for j in range(Wn):
                data = x[:, i * stride[0]:i * 1 + HH, j * stride[1]:j * 1 + WW, :]
                filt = w[m,:,:,:]
                Y[:, i, j, m].assign(tf.reduce_sum(tf.multiply(data,filt),axis=(1,2,3))+b[m])

    return Y

In [5]:
a=corr2d(x,w,b,pad,stride)

In [6]:
a.shape

TensorShape([10, 28, 28, 6])

In [7]:
# help(tf.keras.layers.Conv2D)

In [9]:
#采用tf自带的卷积层
aa=tf.keras.layers.Conv2D(filters=6,kernel_size=(3,3))(x)
print(aa.shape)

(10, 26, 26, 6)


### 手动实现池化层
可以与自带的tf.keras.maxPool2D比较一下

注意：经过池化层，只有尺寸维度会发生变化

In [31]:
def pool2d(x, pool_size=(2,2), stride=(1,1)):
    N, H, W, C = tf.shape(x)
    pool_h,pool_w=pool_size

    Hn = (H-pool_h+1)// stride[0]
    Wn = (W -pool_w+1) // stride[1]
    print(Hn, Wn)
    Y = tf.Variable(tf.zeros((N, Hn, Wn, C),dtype=tf.float32))

    for i in range(Hn):
        for j in range(Wn):
                data = x[:, i * stride[0]:(i * stride[0]+pool_h), j * stride[1]:(j * stride[1]+pool_w), :]
                Y[:, i, j, :].assign(tf.reduce_max(data,axis=(1,2)))

    return Y

In [32]:
bb=pool2d(aa)

tf.Tensor(25, shape=(), dtype=int32) tf.Tensor(25, shape=(), dtype=int32)


In [30]:
bb.shape

TensorShape([10, 25, 25, 6])

In [34]:
cc = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(aa)
print(cc.shape)

(10, 25, 25, 6)


### 手动实现全连接层

全连接层自定义代码可以参见TF3.1。当时没有注意到全连接层其实就是一线性加权 input*w+B=output，其中w和B都是随机生成的

In [35]:
#Dense 全连接层
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):#**kwargs指任意数量的传入参数
        self.units = units
        super(MyDense, self).__init__(**kwargs)#MyDense继承父类tf.keras.layers.Layer中的__init__定义与参数

    #build方法一般定义Layer需要被训练的参数。    
    def build(self, input_shape): 
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='w')
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='b')
        super(MyDense,self).build(input_shape) # 相当于设置self.built = True

    #call方法一般定义正向传播运算`逻辑，__call__方法调用了它。    
    def call(self, inputs): 
        return tf.matmul(inputs, self.w) + self.b

    #如果要让自定义的Layer通过Functional API 组合成模型时可以序列化，需要自定义get_config方法。
    def get_config(self):  
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})##???这个units从哪里来的呢
        return config